In [1]:
import pathpy as pp

ImportError: cannot import name 'Network' from 'pathpy.models.network' (c:\users\ingos\documents\workbench\pathpy3\pathpy\models\network.py)

The following code generates a simple example instance for a set of paths. We use a custum attribute `frequency` to store the observed frequencies of each path. Right now, the construction of paths is based on an edge sequence. The interface is more cumbersome than it needs to be, and we are currently working on simplifying the API of the `Path` and `PathCollection` classes. There should be a better version by next week.

In [4]:
pc = pp.PathCollection()
a = pp.Node('a')
b = pp.Node('b')
c = pp.Node('c')
d = pp.Node('d')

e1 = pp.Edge(a, b, uid='a-b')
e2 = pp.Edge(b, c, uid='b-c')
e3 = pp.Edge(c, d, uid='c-d')

# 15 observations of path a-b
# this path of length one should be ignored as there's no associated state node (empty memory)
pc.add(pp.Path(e1, frequency=15))

# 42 observations of path a-b-c
# this leads to state nodes a-b and b-c connected by a link with weight 42
pc.add(pp.Path(e1, e2, frequency=42))

# 41 observations of path a-b-c-d
# this leads to state nodes a-b-c and b-c-d connected by a link with weight 41
pc.add(pp.Path(e1, e2, e3, frequency=41))
print(pc)

{<pathpy.core.path.Path object at 0x7fb808f5fd90>, <pathpy.core.path.Path object at 0x7fb808f5fdc0>, <pathpy.core.path.Path object at 0x7fb808f5fe50>}


In [6]:
pp.io.infomap.to_state_file(pc, 'test.state', weight='frequency')
with open('test.state', 'r') as f:
    print(f.read())

*Vertices 4
1 "c"
2 "a"
3 "b"
4 "d"
*States
1 3 "{a}_b"
2 1 "{b}_c"
3 1 "{a-b}_c"
4 4 "{b-c}_d"
*Links
1 2 42
3 4 41
